<a href="https://colab.research.google.com/github/basse058/billboard_v3/blob/main/data_S3_to_RDS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Activate Spark in our Colab notebook.
import os
# Find the latest version of spark 3.2  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.2.2'
spark_version = 'spark-3.2.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease [1,581 B]
Hit:2 http://archive.ubuntu.com/ubuntu focal InRelease
Get:3 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease [3,622 B]
Get:4 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Get:5 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Get:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease [18.1 kB]
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  Packages [920 kB]
Get:8 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Hit:10 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Get:11 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 Packages [3,021 kB]
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease
Hit:13 http://ppa.launchpad.net/ubuntugis/ppa/ubuntu fo

In [ ]:
# Get postgresql package
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2023-03-17 18:31:41--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  5.49MB/s    in 0.2s    

2023-03-17 18:31:41 (5.49 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



In [ ]:
# Import Spark and create a SparkSession
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("billboard-100").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

# Extract the Amazon Data into Spark DataFrame

In [ ]:
# Read in the data from an S3 Bucket
from pyspark import SparkFiles

url = "https://billboard-top-100-project-4.s3.amazonaws.com/billboard_songs.csv" #enter correct address here

spark.sparkContext.addFile(url)  

df = spark.read.option('header', 'true').csv(SparkFiles.get("billboard_songs.csv"), inferSchema=True, sep=',')

df.show(10)


+--------------------+--------------------+----+---------+--------------+--------------------+------------+------+----+--------+----+-----------+------------+----------------+--------+-------+-------+-----------+--------------+---------+------+---+
|                song|              artist|year|peak-rank|weeks-on-board|            track_id|danceability|energy| key|loudness|mode|speechiness|acousticness|instrumentalness|liveness|valence|  tempo|duration_ms|time_signature|billboard|decade| id|
+--------------------+--------------------+----+---------+--------------+--------------------+------------+------+----+--------+----+-----------+------------+----------------+--------+-------+-------+-----------+--------------+---------+------+---+
|(1-2-3-4-5-6-7) C...| Inez & Charlie Foxx|1968|       76|             5|4wXNtDvLZl55vlQas...|       0.686| 0.562| 5.0|  -8.538| 1.0|     0.0504|       0.589|        5.34e-05|   0.124|  0.793|116.782|   156886.0|           4.0|      1.0| 1960s|  0|
|(Al

In [ ]:
print(df.count())
df1 = df.na.drop(subset=["id"])
print(df1.count())
df2 = df1.dropDuplicates(subset=["id"])
print(df2.count())

1225740
1217668
1216065


In [ ]:
df2.printSchema()

root
 |-- song: string (nullable = true)
 |-- artist: string (nullable = true)
 |-- year: integer (nullable = true)
 |-- peak-rank: integer (nullable = true)
 |-- weeks-on-board: string (nullable = true)
 |-- track_id: string (nullable = true)
 |-- danceability: float (nullable = true)
 |-- energy: float (nullable = true)
 |-- key: float (nullable = true)
 |-- loudness: float (nullable = true)
 |-- mode: float (nullable = true)
 |-- speechiness: float (nullable = true)
 |-- acousticness: float (nullable = true)
 |-- instrumentalness: float (nullable = true)
 |-- liveness: float (nullable = true)
 |-- valence: float (nullable = true)
 |-- tempo: float (nullable = true)
 |-- duration_ms: float (nullable = true)
 |-- time_signature: float (nullable = true)
 |-- billboard: integer (nullable = true)
 |-- decade: string (nullable = true)
 |-- id: integer (nullable = true)



In [ ]:
from pyspark.sql.functions import col

df2 = df2.withColumn("peak-rank",col("peak-rank").cast("int"))
df2 = df2.withColumn("year",col("year").cast("int"))
df2 = df2.withColumn("billboard",col("billboard").cast("int"))
df2 = df2.withColumn("danceability",col("danceability").cast("float"))
df2 = df2.withColumn("energy",col("energy").cast("float"))
df2 = df2.withColumn("key",col("key").cast("float"))
df2 = df2.withColumn("loudness",col("loudness").cast("float"))
df2 = df2.withColumn("mode",col("mode").cast("float"))
df2 = df2.withColumn("speechiness",col("speechiness").cast("float"))
df2 = df2.withColumn("acousticness",col("acousticness").cast("float"))
df2 = df2.withColumn("instrumentalness",col("instrumentalness").cast("float"))
df2 = df2.withColumn("liveness",col("liveness").cast("float"))
df2 = df2.withColumn("valence",col("valence").cast("float"))
df2 = df2.withColumn("tempo",col("tempo").cast("float"))
df2 = df2.withColumn("duration_ms",col("duration_ms").cast("float"))
df2 = df2.withColumn("time_signature",col("time_signature").cast("float"))
df2 = df2.withColumn("id",col("id").cast("int"))

In [ ]:
df2 = df2.withColumnRenamed("peak-rank", "peak_rank") \
         .withColumnRenamed("weeks-on-board", "weeks_on_board") \
         .withColumnRenamed("key", "music_key") \
         .withColumnRenamed("mode", "music_mode") \
         .withColumnRenamed("year", "release_year")
df2.show(5)

+--------------------+--------------------+------------+---------+--------------+--------------------+------------+------+---------+--------+----------+-----------+------------+----------------+--------+-------+-------+-----------+--------------+---------+------+---+
|                song|              artist|release_year|peak_rank|weeks_on_board|            track_id|danceability|energy|music_key|loudness|music_mode|speechiness|acousticness|instrumentalness|liveness|valence|  tempo|duration_ms|time_signature|billboard|decade| id|
+--------------------+--------------------+------------+---------+--------------+--------------------+------------+------+---------+--------+----------+-----------+------------+----------------+--------+-------+-------+-----------+--------------+---------+------+---+
|(All Of A Sudden)...|          Mel Carter|        1965|       38|             7|6x5BiQwNlbtisITsE...|       0.419| 0.546|      7.0|  -7.819|       1.0|     0.0363|       0.479|         4.81E-6|  

In [ ]:
# Get the number of rows in the DataFrame.
df2 = df2.withColumn("weeks_on_board",col("weeks_on_board").cast("int"))
df2.printSchema()


root
 |-- song: string (nullable = true)
 |-- artist: string (nullable = true)
 |-- release_year: integer (nullable = true)
 |-- peak_rank: integer (nullable = true)
 |-- weeks_on_board: integer (nullable = true)
 |-- track_id: string (nullable = true)
 |-- danceability: float (nullable = true)
 |-- energy: float (nullable = true)
 |-- music_key: float (nullable = true)
 |-- loudness: float (nullable = true)
 |-- music_mode: float (nullable = true)
 |-- speechiness: float (nullable = true)
 |-- acousticness: float (nullable = true)
 |-- instrumentalness: float (nullable = true)
 |-- liveness: float (nullable = true)
 |-- valence: float (nullable = true)
 |-- tempo: float (nullable = true)
 |-- duration_ms: float (nullable = true)
 |-- time_signature: float (nullable = true)
 |-- billboard: integer (nullable = true)
 |-- decade: string (nullable = true)
 |-- id: integer (nullable = true)



In [ ]:
print(df2.count())

df3 = df2.na.drop(subset=["id"])
print(df3.count())

1216065
1216065


# Transform the Data

# Load

In [ ]:
mode = "append"
jdbc_url="jdbc:postgresql://billboard-db.c4q3joupwllm.us-east-1.rds.amazonaws.com:5432/project-4"
config = {"user":"postgres", "password": "testpass", "driver":"org.postgresql.Driver"}

In [ ]:
# Write review_id_df to table in RDS
df3.write.jdbc(url=jdbc_url, table='data_table', mode=mode, properties=config)